## Analyzing Energy Contributions
* https://openmm.github.io/openmm-cookbook/latest/notebooks/cookbook/Analyzing%20Energy%20Contributions.html
* Decompose the energy of a system into separate components.
    * bonds, angles, torsions, non-bonded interactions, etc.


In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *

In [2]:
ala_pdb = PDBFile('./data/ala_ala_ala.pdb')
forcefield = ForceField('amber14-all.xml')
system = forcefield.createSystem(ala_pdb.topology)

* Since openmm does not provide a built-in way to calculate the energy of a single component.
* but can query the energy of a force group.
* We need each Force object to be assigned to a different force group.

In [5]:
for i, f in enumerate(system.getForces()):
    print(i, f.__class__.__name__)
    f.setForceGroup(i)

0 HarmonicBondForce
1 PeriodicTorsionForce
2 NonbondedForce
3 CMMotionRemover
4 HarmonicAngleForce


In [6]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(ala_pdb.topology, system, integrator, platform)
simulation.context.setPositions(ala_pdb.positions)

* We can use "Context.getState(getEnergy=True, groups={force_group})" to get the energy of a single component.


In [7]:
for i, f in enumerate(system.getForces()):
    state = simulation.context.getState(getEnergy=True, groups={i})
    print(f.getName(), state.getPotentialEnergy())

HarmonicBondForce 18.088199615478516 kJ/mol
PeriodicTorsionForce 84.3203125 kJ/mol
NonbondedForce -130.9884033203125 kJ/mol
CMMotionRemover 0.0 kJ/mol
HarmonicAngleForce 70.43386840820312 kJ/mol


In [8]:
merged_H_pdb = PDBFile('./results/merged_H.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(merged_H_pdb.topology)

In [10]:
for i, f in enumerate(system.getForces()):
    print(i, f.getName())
    f.setForceGroup(i)

0 HarmonicBondForce
1 NonbondedForce
2 PeriodicTorsionForce
3 CMMotionRemover
4 HarmonicAngleForce


In [13]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

simulation = Simulation(merged_H_pdb.topology, system, integrator, platform)
simulation.context.setPositions(merged_H_pdb.positions)

In [14]:
for i, f in enumerate(system.getForces()):
    state = simulation.context.getState(getEnergy=True, groups={i})
    print(f.getName(), state.getPotentialEnergy())

HarmonicBondForce 560.353141784668 kJ/mol
NonbondedForce 53661662.3873291 kJ/mol
PeriodicTorsionForce 1980.8448181152344 kJ/mol
CMMotionRemover 0.0 kJ/mol
HarmonicAngleForce 1332.1212158203125 kJ/mol
